In [ ]:
%pylab
#%load_ext autoreload

import logging
#%autoreload 2
import sfg2d
from sfg2d.raw_reader import *
import sfg2d.ana.pumpProbePlots as pp
models = {}
logging.basicConfig(level=logging.DEBUG)

# Import
## Data

Read records from raw data (slow)

In [ ]:
config_records = read_yaml('./records.yaml')
records = import_records(config_records)
pp.records = records
#cache_records(records)

Read records from cahced data (fast)

In [ ]:
#records = read_cache('./cache/')

## Models

In [ ]:
#config_models = read_yaml('./models.yaml')
#models = make_models(config_models, records, save_models=False)
#pp.models = models

In [ ]:
# Update needed if model changed
pp.records = records
pp.models = models

# Spectra

## Background

In [ ]:
kwargs_xdata = {'prop': 'pixel'} 
kwargs_ydata = {
    'prop': 'basesubed', 
    'frame_med': True,
    'delay_mean': True,
    #'roi_delay': slice(9),
}
record_names = [
    'bg200', 
    'bg200_ts'
    #'bg10'
]

fig, ax = subplots()
plt.title("Background")
pp.plot_spectra(record_names, kwargs_xdata, kwargs_ydata)
plt.xlabel('pixel')
plt.ylabel('counts')
plt.ylim(160, 200)

## Quartz

In [ ]:
kwargs_xdata = {'prop': 'wavenumber'}
kwargs_ydata = {'prop': 'basesubed', 
                #'roi_spectra': [0, 2], 
                'frame_med': True}
record_names = ['q0', 'q1']
record_names2 = ['q0_pl']

fig, ax = subplots(2, 1, sharex=True)
sca(ax[0])
plt.title("Quartz")
pp.plot_spectra(record_names, kwargs_xdata, kwargs_ydata)
plt.ylabel('counts')

sca(ax[1])
plt.title('Leackage of Pump-Vis SFG')
pp.plot_spectra(record_names2, kwargs_xdata, kwargs_ydata)
plt.xlabel('frequency/cm$^{-1}$')
plt.ylabel('counts')
plt.ylim(-5, 30)

## Gold

### CC

In [ ]:
roi_wavenumber = slice(4600, 5200)

kwargs_xdata = {'prop': 'wavenumber', 'roi_wavenumber': roi_wavenumber}
kwargs_ydata = {
    'prop': 'basesubed', 
    'roi_wavenumber': roi_wavenumber, 
    'frame_med': True, 
    #'delay_mean': True,
    #'roi_spectra': [2]
}
record_names = ['cc', 'cc1']

fig, ax = subplots()
plt.title('Spectrum of Crosscorrelation')
pp.plot_spectra(record_names, kwargs_xdata, kwargs_ydata)

# Add Mean Spectrum
kwargs_ydata['delay_mean'] = True
kwargs_plots={
    'cc': {'label': 'Mean', 'marker': 'o'}, 
    'cc1': {'label': 'Mean', 'marker': 'o'}, 
}

pp.plot_spectra(record_names, kwargs_xdata, kwargs_ydata, kwargs_plots=kwargs_plots)
plt.legend()
plt.xlabel('frequency/cm$^{-1}$')
plt.ylabel('counts')

### Pump-Vis

In [ ]:
kwargs_xdata = {'prop': 'wavenumber'}
kwargs_ydata = {'prop': 'basesubed', 'frame_med': True, 
                'delay_mean': True,
                #'roi_spectra': [1],
               }
record_names = ['pump_vis']

fig, ax = subplots()
plt.title('Spectrum of pump-vis')
pp.plot_spectra(record_names, kwargs_xdata, kwargs_ydata)
plt.xlabel('frequency/cm$^{-1}$')
plt.ylabel('counts')

# Plot fit
model_names = ['pump_vis']
text_kwargs = {'pump_vis': {'x': 2600, 'y': 2}}
pp.plot_models(model_names, text_kwargs=text_kwargs)

In [ ]:
#config_models = read_yaml('./models.yaml')
#models = make_models(config_models, records, save_models=False)
#pp.models = models

### Spectrum

In [ ]:
kwargs_xdata = {'prop': 'pixel'}
kwargs_ydata = {'prop': 'basesubed', 'frame_med': True, 'delay_mean': True}
record_names = ['gold']

fig, ax = subplots()
plt.title('Spectrum of Gold')
pp.plot_spectra(record_names, kwargs_xdata, kwargs_ydata)
plt.xlabel('frequency/cm$^{-1}$')
plt.ylabel('counts')

## Samples

In [ ]:
kwargs_xdata = {'prop': 'wavenumber'}
kwargs_ydata = {
    'prop': 'basesubed', 
    'frame_med': True, 
    'delay_mean': True, 
    'roi_spectra': [1],
    #'scale': 1000,
    #'roi_frames': slice(0, 5),
}
record_names = ['d2o', 'na2co3', 'na2so4']
titles = [
    'Spectrum of D$_2$O', 
    'Spectrum of Na$_2$CO$_3$', 
    'Spectrum of Na$_2$SO$_4$',
]

for record_name, title_str in zip(record_names, titles):
    if record_name not in records.keys():
        continue
        
    fig, ax = subplots()
    plt.title(title_str)
    pp.plot_spectra([record_name], kwargs_xdata, kwargs_ydata)
    plt.xlabel('frequency/cm$^{-1}$')
    plt.xlim(2100, 2600)
    #plt.ylim(-2, 20)

# Tracks

In [ ]:
kwargs_ydata = {
    'prop': 'track', 
    'roi_spectra': slice(1), 
    # Choos aggregatoon function for track. 
    'kwargs_prop': {'aggr': np.nanmean}, 
    #'roi_frames': [0, 1, 10],
}
record_names = [
    'd2o', 'na2co3', 'na2so4'
]
kwargs_plots={
    'd2o': {'label': 'd2o', 'marker': 'o'},
    'na2co3': {'label': 'na2co3', 'marker': 'o'},
    'na2so4': {'label': 'na2so4', 'marker': 'o'},

}


fig, ax = subplots()
plt.title('Track')
pp.plot_tracks(record_names, kwargs_ydata, kwargs_plots)
plt.xlabel('number of frame')
plt.ylabel('Mean Counts')
plt.legend()

# Traces

## Gold

### CC

In [ ]:
kwargs_xdata = {'prop': 'pp_delays_ps'}
kwargs_ydata = {
    'prop': 'trace', 
    'kwargs_prop': {'prop': 'basesubed'}, 
    'frame_med': True, 
    'roi_wavenumber': slice(4600, 5200),
    #'roi_spectra': [1],
}
record_names = ['cc']

fig, ax = subplots()
plt.title('Crosscorrelation')
pp.plot_traces(record_names, kwargs_xdata, kwargs_ydata)
plt.xlabel('time/ps')
plt.ylabel('counts')

# Show the fit if possible

# Plot fit
model_names = ['cc']
text_kwargs = {'cc': {'x': -2, 'y': 1000}}
try:
    pp.plot_models(model_names, text_kwargs=text_kwargs)
except KeyError:
    pass

In [ ]:
#config_models = read_yaml('./models.yaml')
#models = make_models(config_models, records, save_models=True)

### Ts

In [ ]:
kwargs_xdata = {'prop': 'pp_delays_ps'}
kwargs_ydata = {
    'prop': 'trace', 
    'frame_med': True, 
    'roi_wavenumber': slice(2300, 2400),
}
record_names = ['gold']

fig, ax = subplots()
plt.title('Trace {} @ {}'.format(record_names[0], kwargs_ydata.get('roi_wavenumber')))
pp.plot_traces(record_names, kwargs_xdata, kwargs_ydata)
plt.xlabel('time/ps')

## Samples

In [ ]:
kwargs_xdata = {'prop': 'pp_delays_ps'}
kwargs_ydata = {
    'prop': 'trace', 
    'frame_med': True, 
    'roi_wavenumber': slice(2420, 2480),
    #'roi_frames': slice(-2,None),
}
record_names = ['na2so4', 'd2o', 'na2co3']
model_names = {
    'na2so4': ['na2so4'],
    'd2o': ['d2o'],
    'na2co3': ['na2co3'],
}

for record_name, model_name in zip(record_names, model_names):
    if record_name not in records.keys():
        continue
    rw = kwargs_ydata.get('roi_wavenumber', slice(None))
    fig, ax = subplots()
    plt.title('Trace {} @ {}-{}'.format(record_name, rw.start, rw.stop))
    pp.plot_traces([record_name], kwargs_xdata, kwargs_ydata)
    plt.xlabel('time/ps')
    plt.xlim(-1, 10)

    # Show fit if possible
    try:
        this_records_model_names = model_names[record_name]
        for model_name in this_records_model_names:
            m = models[model_name]
            plt.plot(m.xsample, m.ysample)
        plt.text(2, 0.9, m.box_str);
    except:
        pass

In [ ]:
#config_models = read_yaml('./models.yaml')
#models = make_models(config_models, records, save_models=True)

# Contour Plot

In [ ]:
record_names = ['d2o', 'na2co3', 'na2so4']
kwargs_data = dict(
    kwargs_xdata = {
        'prop': 'pp_delays_ps'
    },
    kwargs_ydata = {
        #'roi_pixel': slice(700, 1150)
        'roi_wavenumber': slice(2200, 2600),
    },
    kwargs_zdata = {
        'medfilt_pixel': 13,
        'resample_freq': 5,
    }
)
kwargs_contourfs = {
    'd2o':{'levels': linspace(0.55, 1.15)},
    'na2co3': {},
    'na2so4': {'levels': linspace(0.7, 1.15)},
}
titles = {'d2o': 'D$_2$O', 'na2so4': 'Na$_2$SO$_4$', 'na2co3': 'Na$_2$CO$_3$'}


for record_name in record_names:
    record = records.get(record_name)
    kwargs_contourf = kwargs_contourfs.get(record_name, {})
    if not record:
        continue
    pp.plot_contour(
        record_name, 
        kwargs_data,
        kwargs_contourf,
    )
    plt.title('{} @ {} 1/cm'.format(titles[record_name], record.pump_freq))
    plt.xlim(-1, 10)

# Figs

## Save

In [ ]:
figs = [plt.figure(i) for i in plt.get_fignums()]
sfg2d.fig.save_figs_to_multipage_pdf(figs, './figures.pdf')

## Close All Figures

In [ ]:
plt.close('all')

# Bleach

In [ ]:
record_names = ['d2o', 'na2so4', 'na2co3']
kwargs_xdata = {
    'prop': 'wavenumber',
    'roi_wavenumber': slice(2200, 2600),
}
kwargs_ydata = {
    'prop': 'bleach', 
    'frame_med': True, 
    #'roi_delay': slice(0, 1),
    'roi_wavenumber': slice(2200, 2600),
    'medfilt_pixel': 5,
    'resample_freqs': 20,
    #'roi_frames': slice(0, 8),
}
sfiles = {
    'd2o': 'bleach_d2o.pdf', 
    'na2so4': 'bleach_na2so4.pdf', 
    'na2co3': 'bleach_na2co3.pdf'
}

for record_name in record_names:
    record = records.get(record_name)
    sfile = sfiles.get(record_name)
    if not record:
        continue
    titles = ['{} @ {} ps'.format(record_name, i) for i in record.pp_delays_ps]
    pp.multifig_bleach(record_name, kwargs_xdata, kwargs_ydata, ylim=(0.5, 1.2), titles=titles, sfile=sfile)

In [ ]:
record_names = ['d2o', 'na2so4', 'na2co3']
kwargs_xdata = {
    'prop': 'wavenumber',
    'roi_wavenumber': slice(2000, 2800),
}
kwargs_ydata = {
    'prop': 'basesubed', 
    'frame_med': True, 
    #'roi_delay': slice(0, 1),
    'roi_wavenumber': slice(2000, 2800),
    'medfilt_pixel': 7,
    'resample_freqs': 30,
    #'roi_frames': slice(0, 8),
}
sfiles = {
    'd2o': 'basesubed_d2o.pdf', 
    'na2so4': 'basesubed_na2so4.pdf', 
    'na2co3': 'basesubed_na2co3.pdf'
}

for record_name in record_names:
    record = records.get(record_name)
    sfile = sfiles.get(record_name)
    if not record:
        continue
    titles = ['{} @ {} ps'.format(record_name, i) for i in record.pp_delays_ps]
    pp.multifig_bleach(
        record_name, kwargs_xdata, kwargs_ydata, #ylim=(0.5, 1.2), 
        titles=titles, sfile=sfile
    )